In [2]:
pip install emoji


   ---------------------------------------- 0.0/608.4 kB ? eta -:--:--
   ------------- -------------------------- 204.8/608.4 kB 4.1 MB/s eta 0:00:01
   ----------------------------- ---------- 450.6/608.4 kB 4.7 MB/s eta 0:00:01
   ---------------------------------------  604.2/608.4 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 608.4/608.4 kB 3.2 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding


from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer


In [6]:
data = pd.read_csv('emoji_data.csv', header = None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [7]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])

In [8]:
X = data[0].values
Y = data[1].values

In [36]:
print(X)

['French macaroon is so tasty' 'work is horrible' 'I am upset'
 'throw the ball' 'Good joke' 'what is your favorite baseball game'
 'I cooked meat' 'stop messing around' 'I want chinese food'
 'Let us go play baseball' 'you are failing this exercise'
 'yesterday we lost again' 'Good job' 'ha ha ha it was so funny'
 'I will have a cheese cake' 'Why are you feeling bad' 'I want to joke'
 'I never said yes for this' 'the party is cancelled' 'where is the ball'
 'I am frustrated' 'ha ha ha lol' 'she said yes' 'he got a raise'
 'family is all I have' 'he can pitch really well'
 'I love to the stars and back' 'do you like pizza '
 'You totally deserve this prize' 'I miss you so much'
 'I like your jacket ' 'she got me a present' 'will you be my valentine'
 'you failed the midterm' 'Who is down for a restaurant'
 'valentine day is near' 'Great so awesome' 'do you have a ball'
 'he can not do anything' 'he likes baseball'
 'We had such a lovely dinner tonight' 'vegetables are healthy'
 'he is 

In [9]:
#Embeddings
file = open('glove.6B.100d.txt', 'r', encoding = 'utf8')
content = file.readlines()
file.close()

In [10]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [1]:
print(word2index)

NameError: name 'word2index' is not defined

In [14]:
Xtokens = tokenizer.texts_to_sequences(X)
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

maxlen = get_maxlen(Xtokens)
print(maxlen)
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')

10


In [21]:
#arr = np.where(arr == "apple", "orange", arr)
Y=np.where(Y=='0v2',2,Y)
#Y.replace('0v2',2,inplace=True)
Ytrain = to_categorical(Y)

# Model

In [22]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))

for word, i in word2index.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [23]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.046539,  0.61966 ,  0.56647 , ..., -0.37616 , -0.032502,
         0.8062  ],
       [-0.49886 ,  0.76602 ,  0.89751 , ..., -0.41179 ,  0.40539 ,
         0.78504 ],
       ...,
       [-0.46263 ,  0.069864,  0.69095 , ..., -0.29174 ,  0.32041 ,
         0.21202 ],
       [ 0.073242,  0.11134 ,  0.62281 , ...,  0.53417 , -0.1646  ,
        -0.27516 ],
       [ 0.29019 ,  0.80497 ,  0.31187 , ..., -0.33603 ,  0.45998 ,
        -0.11278 ]])

In [24]:
model = Sequential([
    Embedding(input_dim = len(word2index) + 1,
              output_dim = embed_size,
              input_length = maxlen,
              weights = [embedding_matrix],
              trainable = False
             ),
    
    LSTM(units = 16, return_sequences = True),
    LSTM(units = 4),
    Dense(5, activation = 'softmax')
])


c:\Users\USER\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [25]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [26]:
model.fit(Xtrain, Ytrain, epochs = 100)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.1596 - loss: 1.6248
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2799 - loss: 1.5825
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2804 - loss: 1.5737 
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3054 - loss: 1.5477
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2904 - loss: 1.5358
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3626 - loss: 1.4964 
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3219 - loss: 1.4975
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3158 - loss: 1.4850 
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3836 - loss: 1.4624 
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4548 - loss: 1.4141 
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5232 - loss: 1.3927
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5229 - los

In [ ]:
test = ["I feel good", "I feel very bad", "lets eat dinner","I love you","He hit a home run"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step
I feel good 😃
I feel very bad 😞
lets eat dinner 🍽️
I love you ❤️
He hit a home run ⚾
She is so cute ❤️
I am very sad ❤️
crazy about baseball ⚾


In [37]:
model.save("emoji_lstm_model.h5")

In [38]:
import pickle

with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)